In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
df = pd.read_csv("~/Downloads/toy_data.csv")

In [3]:
# df.dropna(axis = 1, thresh= 0.9*df.shape[0], inplace= True)
# nunique = df.apply(pd.Series.nunique)
# cols_to_drop = nunique[nunique == 1].index
# df.drop(cols_to_drop, axis = 1, inplace= True)

In [4]:
#required_cols = ['agency_name','agency_abbr','agency_code','state_name','state_abbr','state_code','loan_amount_000s', 'applicant_sex_name', 'applicant_sex', 'tract_to_msamd_income', 'population','census_tract_number','number_of_1_to_4_family_units','minority_population', 'applicant_income_000s']

In [5]:
required_cols = ['applicant_sex', 'applicant_income_000s', 'applicant_sex_name','agency_abbr']

In [6]:
df = df[required_cols]

In [7]:
nunique = df.apply(pd.Series.nunique)
discrete_columns = nunique[nunique < 100].index

In [8]:
numeric_columns = df.select_dtypes(include=['number']).columns
object_columns = df.select_dtypes(include=['object']).columns
df[numeric_columns] = df[numeric_columns].fillna(0)
df[object_columns] = df[object_columns].fillna("NA")

df.isna().any()

applicant_sex            False
applicant_income_000s    False
applicant_sex_name       False
agency_abbr              False
dtype: bool

In [26]:
temp = ['applicant_sex','agency_abbr']

In [29]:
from importlib import reload
import synthesizer,transformer,sampler,conditional,models
reload(models)
reload(sampler)
reload(synthesizer)
reload(transformer)
reload(conditional)
data = df.loc[0:100000,:]
model = synthesizer.CTGANSynthesizer(embedding_dim = 128, batch_size=250)
model.fit(data, discrete_columns, temp, 40,log_frequency= False)

Umbrella Sampling for column: applicant_sex
    [d]: Total windows: 8
    [d]: Temperatures: [ 1.          2.28571429  3.57142857  4.85714286  6.14285714  7.42857143
  8.71428571 10.        ]
 :- Completed 1000 of 1000 iterations.
    [d]: max gr: 0.04570333756022982
    [d]: Repex summary (62.85 %):  [ 0.  0.  6. 10.  9. 10.  9.  0.]
    [d]: z values [5.53034460e-10 5.33189657e-08 2.35926095e-05 9.57748937e-04
 1.12454222e-02 6.55661353e-02 2.44901580e-01 6.77305467e-01]
    [d]: Acor values [1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
    [d]: Percentage burned: 50.0 % (32000 of 64000 )
Umbrella Sampling for column: applicant_sex_name
    [d]: Total windows: 8
    [d]: Temperatures: [ 1.          2.28571429  3.57142857  4.85714286  6.14285714  7.42857143
  8.71428571 10.        ]
 :- Completed 1000 of 1000 iterations.
    [d]: max gr: 0.0336502009485542
    [d]: Repex summary (60.0 %):  [ 0.  0. 10. 10.  7.  7.  8.  0.]
    [d]: z values [9.53678590e-11 2.6583480

In [72]:
sampled = model.sample(100000,col_index= [6])

tensor(2.5913, grad_fn=<DivBackward0>)
tensor(2.8935, grad_fn=<DivBackward0>)
tensor(2.6942, grad_fn=<DivBackward0>)
tensor(2.7973, grad_fn=<DivBackward0>)
tensor(2.8171, grad_fn=<DivBackward0>)
tensor(2.6740, grad_fn=<DivBackward0>)
tensor(3.0205, grad_fn=<DivBackward0>)
tensor(3.0286, grad_fn=<DivBackward0>)
tensor(2.6713, grad_fn=<DivBackward0>)
tensor(2.6725, grad_fn=<DivBackward0>)
tensor(2.8839, grad_fn=<DivBackward0>)
tensor(2.7911, grad_fn=<DivBackward0>)
tensor(3.0846, grad_fn=<DivBackward0>)
tensor(2.7786, grad_fn=<DivBackward0>)
tensor(2.5981, grad_fn=<DivBackward0>)
tensor(2.7742, grad_fn=<DivBackward0>)
tensor(2.8074, grad_fn=<DivBackward0>)
tensor(2.6998, grad_fn=<DivBackward0>)
tensor(2.9507, grad_fn=<DivBackward0>)
tensor(2.9014, grad_fn=<DivBackward0>)
tensor(2.8748, grad_fn=<DivBackward0>)
tensor(2.9879, grad_fn=<DivBackward0>)
tensor(2.9750, grad_fn=<DivBackward0>)
tensor(2.7621, grad_fn=<DivBackward0>)
tensor(2.6446, grad_fn=<DivBackward0>)
tensor(2.7387, grad_fn=<D

In [73]:
sampled["applicant_sex_name"][sampled.applicant_sex_name == "Information not provided by applicant in mail, Internet, or telephone application"] = "Not Provided"
data["applicant_sex_name"][data.applicant_sex_name == "Information not provided by applicant in mail, Internet, or telephone application"] = "Not Provided"
agency_abbr_col_reindex = np.array(["CFPB", "HUD", "FDIC", "FRS", "NCUA", "OCC"])
applicant_sex_col_reindex = np.array([2, 1, 3,4])


In [74]:
def calculate_contingency_table(data, feature1, feature2, col_reindex = None):
    og = (pd.crosstab(data[feature1], data[feature2])/data.shape[0] * 100)
    if col_reindex != None:
        og = og.reindex(col_reindex, axis="columns")
    og['Total'] = og.sum(axis=1)
    og.loc['Total']= og.sum()
    og = og.round(2)
    return og

In [75]:
feature1 = 'applicant_sex_name'
feature2 = 'agency_abbr'
col_reindex = applicant_sex_col_reindex

In [76]:
calculate_contingency_table(data, feature1, feature2)

agency_abbr,CFPB,FDIC,FRS,HUD,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,6.00,2.09,0.79,14.84,2.36,0.97,27.06
Male,13.38,4.96,1.82,31.63,4.08,2.39,58.28
Not Provided,3.30,0.97,0.32,8.36,1.26,0.41,14.62
Not applicable,0.01,0.01,0.00,0.01,0.01,0.00,0.05
Total,22.70,8.04,2.94,54.84,7.70,3.78,100.00


In [77]:
calculate_contingency_table(sampled, feature1, feature2)

agency_abbr,CFPB,FDIC,FRS,HUD,NCUA,OCC,Total
applicant_sex_name,,,,,,,
Female,0.24,0.07,0.03,0.78,0.05,0.04,1.22
Male,19.00,6.07,1.94,63.73,3.22,3.30,97.28
Not Provided,0.19,0.06,0.02,0.60,0.03,0.02,0.93
Not applicable,0.10,0.04,0.01,0.37,0.02,0.02,0.57
Total,19.54,6.25,2.01,65.48,3.32,3.40,100.00


In [23]:
#import synthesizer,transformer,sampler,conditional,models,data
torch.save(model, "/Users/sahanalva/Counterfactual Research/usample_test.pt")

In [18]:
k = {2:1,3:3,5:7}
if k:
    print("ji")

ji


In [22]:
np.random.choice(list(k.keys()))

2